## Ading New Features

### Importing Libraries and Cleaned data

In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '/home/jabez/Documents/week_8/Logistic-optimization-Delivery-drivers-location-optimisation-with-Causal-Inference-/scripts'))
import eda
import adding_feature



In [45]:
df = pd.read_csv('/home/jabez/Documents/week_8/Logistic-optimization-Delivery-drivers-location-optimisation-with-Causal-Inference-/data/df_concatenated.csv')
df = df.drop('Unnamed: 0', axis=1)
df_weather = pd.read_csv('/home/jabez/Documents/week_8/Logistic-optimization-Delivery-drivers-location-optimisation-with-Causal-Inference-/data/lagos 2021-01-01 to 2022-12-31.csv')
df_weather=df_weather[['datetime','icon']]

In [46]:
df_saved = pd.read_csv('/home/jabez/Documents/week_8/Logistic-optimization-Delivery-drivers-location-optimisation-with-Causal-Inference-/data/trip.csv')
df_saved.rename(columns={'Trip ID': 'order_id'}, inplace=True)
df_holiday = df_saved[['order_id', 'holiday_name']]
df_holiday.head(5)

,order_id,holiday_name
0,391996,No holiday
1,391997,No holiday
2,391998,No holiday
3,391999,No holiday
4,392001,No holiday


In [47]:
df = df.merge(df_holiday, on='order_id', how='right')
df.head(5)

,order_id,lat_org,lon_org,lat_des,lon_des,date,driver_action,holiday_name
0,391996,6.508813,3.377403,6.650970,3.345031,2021-07-01,accepted,No holiday
1,391997,6.431671,3.455538,6.428081,3.472189,2021-07-01,accepted,No holiday
2,391998,6.631679,3.338898,6.508324,3.359040,2021-07-01,accepted,No holiday
3,391999,6.572757,3.367708,6.584881,3.361407,2021-07-01,accepted,No holiday
4,392001,6.601042,3.276634,6.450107,3.391615,2021-07-01,accepted,No holiday


In [48]:
df.isnull().sum()

order_id             0
lat_org          26038
lon_org          26038
lat_des          26038
lon_des          26038
date             26038
driver_action    26038
holiday_name         0
dtype: int64

In [49]:
df = eda.drop_missing_values(df, threshold=0.05)

In [50]:
df.isnull().sum()

order_id         0
lat_org          0
lon_org          0
lat_des          0
lon_des          0
date             0
driver_action    0
holiday_name     0
dtype: int64

### Geting Holidays from calendarific

In [ ]:
import requests

# Function to get holiday name for a specific date
def get_holiday_name(row):
    year = str(row['st_year'])
    month = str(row['st_month'])
    day = str(row['st_date'])
    api_key = ''
    url = f'https://calendarific.com/api/v2/holidays?&api_key={api_key}&country=NG&year={year}&month={month}&day={day}'
    response = requests.get(url)
    data = response.json()
    
    try:
        # Check if 'holidays' key exists and it's not empty
        if 'holidays' in data['response'] and data['response']['holidays']:
            holiday_name = data['response']['holidays'][0]['name']
        else:
            holiday_name = "No holiday"
    except (KeyError, IndexError):
        # Handle KeyError (if 'response' or 'name' key is missing) or IndexError (if 'holidays' list is empty)
        holiday_name = "No holiday"
    
    return holiday_name

# Extract unique dates from the DataFrame
unique_dates = df[['st_year', 'st_month', 'st_date']].drop_duplicates()

# Get holiday names for these unique dates
unique_dates['holiday_name'] = unique_dates.apply(get_holiday_name, axis=1)

# Create a dictionary for mapping
date_to_holiday = {
    (row['st_year'], row['st_month'], row['st_date']): row['holiday_name']
    for _, row in unique_dates.iterrows()
}

# Map the holiday names back to the original DataFrame
df['holiday_name'] = df.apply(lambda row: date_to_holiday[(row['st_year'], row['st_month'], row['st_date'])], axis=1)

# Display the DataFrame
print(df)


### Adding Enviromental Data 

In [51]:
df_weather.rename(columns={'datetime': 'date'}, inplace=True)
df = pd.merge(df_weather,df, on='date')
df.head(5)

,date,icon,order_id,lat_org,lon_org,lat_des,lon_des,driver_action,holiday_name
0,2021-07-01,rain,391996,6.508813,3.377403,6.650970,3.345031,accepted,No holiday
1,2021-07-01,rain,391997,6.431671,3.455538,6.428081,3.472189,accepted,No holiday
2,2021-07-01,rain,391998,6.631679,3.338898,6.508324,3.359040,accepted,No holiday
3,2021-07-01,rain,391999,6.572757,3.367708,6.584881,3.361407,accepted,No holiday
4,2021-07-01,rain,392001,6.601042,3.276634,6.450107,3.391615,accepted,No holiday


### Adding Weekday/Weekend data

In [54]:
# Apply the function to create a new column
df['date'] = pd.to_datetime(df['date'])
df['day_type'] = df['date'].apply(adding_feature.determine_weekday_or_weekend)

df

,date,icon,order_id,lat_org,lon_org,lat_des,lon_des,driver_action,holiday_name,day_type
0,2021-07-01,rain,391996,6.508813,3.377403,6.650970,3.345031,accepted,No holiday,Weekday
1,2021-07-01,rain,391997,6.431671,3.455538,6.428081,3.472189,accepted,No holiday,Weekday
2,2021-07-01,rain,391998,6.631679,3.338898,6.508324,3.359040,accepted,No holiday,Weekday
3,2021-07-01,rain,391999,6.572757,3.367708,6.584881,3.361407,accepted,No holiday,Weekday
4,2021-07-01,rain,392001,6.601042,3.276634,6.450107,3.391615,accepted,No holiday,Weekday
...,...,...,...,...,...,...,...,...,...,...
2047433,2022-05-19,rain,1345209,6.635022,3.338230,6.460184,3.193621,accepted,No holiday,Weekday
2047434,2022-05-30,partly-cloudy-day,1604908,6.509019,3.387048,6.626393,3.325983,accepted,No holiday,Weekday
2047435,2022-06-03,rain,1488991,6.444520,3.547642,6.606708,3.318793,accepted,No holiday,Weekday
2047436,2022-06-04,partly-cloudy-day,1420310,6.452809,3.433489,6.557231,3.290836,accepted,No holiday,Weekend


### Calculating distance between driver and customer

In [55]:
df['distance_to_client_km'] = df.apply(lambda row: adding_feature.haversine(row['lat_org'], row['lon_org'], row['lat_des'] ,row['lon_des']), axis=1)
df.head(5)

,date,icon,order_id,lat_org,lon_org,lat_des,lon_des,driver_action,holiday_name,day_type,distance_to_client_km
0,2021-07-01,rain,391996,6.508813,3.377403,6.650970,3.345031,accepted,No holiday,Weekday,16.206549
1,2021-07-01,rain,391997,6.431671,3.455538,6.428081,3.472189,accepted,No holiday,Weekday,1.882675
2,2021-07-01,rain,391998,6.631679,3.338898,6.508324,3.359040,accepted,No holiday,Weekday,13.895773
3,2021-07-01,rain,391999,6.572757,3.367708,6.584881,3.361407,accepted,No holiday,Weekday,1.517186
4,2021-07-01,rain,392001,6.601042,3.276634,6.450107,3.391615,accepted,No holiday,Weekday,21.048262


In [57]:
df['z_score'] = (df['distance_to_client_km'] - df['distance_to_client_km'].mean()) / df['distance_to_client_km'].std()

# Identify outliers
outliers = df[np.abs(df['z_score']) > 3]
outliers

,date,icon,order_id,lat_org,lon_org,lat_des,lon_des,driver_action,holiday_name,day_type,distance_to_client_km,z_score
417,2021-07-01,rain,392025,6.425410,3.456893,6.603051,3.267502,accepted,No holiday,Weekday,28.774309,3.026744
420,2021-07-01,rain,392025,6.425588,3.456783,6.603051,3.267502,rejected,No holiday,Weekday,28.751874,3.023278
423,2021-07-01,rain,392025,6.430025,3.468372,6.603051,3.267502,rejected,No holiday,Weekday,29.370434,3.118841
427,2021-07-01,rain,392025,6.430025,3.468372,6.603051,3.267502,rejected,No holiday,Weekday,29.370434,3.118841
2037,2021-07-01,rain,392059,6.637794,3.365564,6.458107,3.599460,accepted,No holiday,Weekday,32.662371,3.627419
...,...,...,...,...,...,...,...,...,...,...,...,...
2047160,2021-12-30,clear-day,1637609,6.637668,3.354650,6.437568,3.560621,accepted,No holiday,Weekday,31.824701,3.498005
2047164,2021-12-30,clear-day,1637624,6.618190,3.372292,6.431633,3.552309,accepted,No holiday,Weekday,28.737128,3.021000
2047211,2021-12-31,partly-cloudy-day,1635943,6.463835,3.556200,6.666710,3.288452,accepted,No holiday,Weekday,37.198006,4.328138
2047219,2021-12-31,partly-cloudy-day,1636565,6.595810,3.353059,6.472755,3.610555,accepted,New Year's Eve,Weekday,31.565987,3.458036


In [61]:
df = df[np.abs(df['z_score']) <= 3]

# Drop the z_score column as it's no longer needed
df = df.drop(columns=['z_score'])

df

,date,icon,order_id,lat_org,lon_org,lat_des,lon_des,driver_action,holiday_name,day_type,distance_to_client_km
0,2021-07-01,rain,391996,6.508813,3.377403,6.650970,3.345031,accepted,No holiday,Weekday,16.206549
1,2021-07-01,rain,391997,6.431671,3.455538,6.428081,3.472189,accepted,No holiday,Weekday,1.882675
2,2021-07-01,rain,391998,6.631679,3.338898,6.508324,3.359040,accepted,No holiday,Weekday,13.895773
3,2021-07-01,rain,391999,6.572757,3.367708,6.584881,3.361407,accepted,No holiday,Weekday,1.517186
4,2021-07-01,rain,392001,6.601042,3.276634,6.450107,3.391615,accepted,No holiday,Weekday,21.048262
...,...,...,...,...,...,...,...,...,...,...,...
2047432,2022-05-19,rain,1117748,6.573463,3.400235,6.539722,3.292317,accepted,No holiday,Weekday,12.497862
2047433,2022-05-19,rain,1345209,6.635022,3.338230,6.460184,3.193621,accepted,No holiday,Weekday,25.162527
2047434,2022-05-30,partly-cloudy-day,1604908,6.509019,3.387048,6.626393,3.325983,accepted,No holiday,Weekday,14.691531
2047436,2022-06-04,partly-cloudy-day,1420310,6.452809,3.433489,6.557231,3.290836,accepted,No holiday,Weekend,19.575610


In [63]:
df.to_csv('/home/jabez/Documents/week_8/Logistic-optimization-Delivery-drivers-location-optimisation-with-Causal-Inference-/data/df_with_added_features.csv')